In [ ]:
import numpy as np
from keras.models import Model, load_model
import gc

## GradCAM
Credits: https://github.com/eclique/keras-gradcam; Last Visited: 23/02/2018
Modified implementation (e.g. decode_predictions(), build_model(), load_model())

In [ ]:
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from keras import backend as K
from keras.preprocessing import image

import tensorflow as tf
from tensorflow.python.framework import ops

from imageio import imread, mimsave

In [ ]:
imgSize=128
H = imgSize
W = imgSize

N_CLASS = 4

### Utility functions

In [ ]:
def build_model(modelPath):
  """Function returning keras model instance.
  """
  model = load_model(modelPath, compile=False)
  return model

In [ ]:
def load_image(path, preprocess=True):
  """Load and preprocess image."""
  if preprocess:
    x = np.array(imread(path),dtype=np.float32)
    x = np.expand_dims(x, axis=0)
    #print(x)
    x/=255. # This was the only preprocessing I did... rather than keras.applications.preprocess_input(x)
  else:
    x = image.load_img(path, target_size=(H, W))
  return x
#def load_image(path, preprocess=True):
#  """Load and preprocess image."""
#  x = image.load_img(path, target_size=(H, W))
#  if preprocess:
#      x = image.img_to_array(x)
#      x = np.expand_dims(x, axis=0)
#      x = preprocess_input(x)
#  return x


def deprocess_image(x):
  """Same normalization as in:
  https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
  """
  x = x.copy()
  if np.ndim(x) > 3:
    x = np.squeeze(x)
  # normalize tensor: center on 0., ensure std is 0.1
  x -= x.mean()
  x /= (x.std() + 1e-5)
  x *= 0.1

  # clip to [0, 1]
  x += 0.5
  x = np.clip(x, 0, 1)

  # convert to RGB array
  x *= 255
  if K.image_dim_ordering() == 'th':
    x = x.transpose((1, 2, 0))
  x = np.clip(x, 0, 255).astype('uint8')
  return x


def normalize(x):
  """Utility function to normalize a tensor by its L2 norm"""
  return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)

In [ ]:
import json
from keras.utils.data_utils import get_file
from keras import backend as K

CLASS_INDEX = json.load(open('./myModel_class_index.json'))
    
def decode_predictions(preds, top=4):
    """Decodes the prediction of ourModel.
    # Arguments
        preds: Numpy tensor encoding a batch of predictions.
        top: Integer, how many top-guesses to return.
    # Returns
        A list of lists of top class prediction tuples
        `(class_name, class_description, score)`.
        One list of tuples per sample in batch input.
    # Raises
        ValueError: In case of invalid shape of the `pred` array
            (must be 2D).
    # Note: this is a modification of keras.applications.imagenet_utils.preprocess_input
    """
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 4:
        raise ValueError('`decode_predictions` expects '
                         'a batch of predictions '
                         '(i.e. a 2D array of shape (samples, 4)). '
                         'Found array with shape: ' + str(preds.shape))
    
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

### Guided Backprop

In [ ]:
def build_guided_model(modelPath):
    """Function returning modified model.
    
    Changes gradient function for all ReLu activations
    according to Guided Backpropagation.
    """
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                   tf.cast(op.inputs[0] > 0., dtype)

    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': 'GuidedBackProp'}):
        new_model = build_model(modelPath)
    return new_model


def guided_backprop(input_model, images, layer_name):
    """Guided Backpropagation method for visualizing input saliency."""
    input_imgs = input_model.input
    layer_output = input_model.get_layer(layer_name).output
    grads = K.gradients(layer_output, input_imgs)[0]
    backprop_fn = K.function([input_imgs, K.learning_phase()], [grads])
    grads_val = backprop_fn([images, 0])[0]
    return grads_val

### GradCAM

In [ ]:
def grad_cam(input_model, image, target_cls, layer_name):
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.output[0, target_cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    # Normalize if necessary
    # grads = normalize(grads)
    gradient_function = K.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Process CAM
    cam = cv2.resize(cam, (H, W), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

# Generate GradCAM across pruning phase

In [ ]:
# Generate commands for the 91 models: {baseModel, 45 x prunedModel, 45 x tunedModel}
# Analyze 3 imgs per class (from test set only). Total 12 imgs (since 4 classes)
# For each img, {GradCAM, GuidedBackprop, GuidedGradCAM} per class. Total 12 CAMs (since 4 classes)


# Selected imgs this way: 
# 1 img that is obviously the target class, the other 2 imgs that are ambiguous (e.g. multiple classes per img or potentially misleading)
img_cat    = ['2008_000536', '2008_000950', '2008_000660']
img_dog    = ['2008_000162', '2008_000829', '2008_001436']
img_horse  = ['2008_000141', '2008_000552', '2008_000880']
img_person = ['2008_000436', '2008_000316', '2008_000277']

## Setup and baseline

In [ ]:
# Make directories if not already exist
categories = ['cat','dog','horse','person']
targetDir = './CAMs/'

for f1 in ['GradCAM', 'G_GradCAM', 'G_BackProp']:
  p1 = '{}{}/'.format(targetDir,f1)
  if not os.path.exists(p1):
    #print('{} path does not exist'.format(p1))
    os.makedirs(p1)
  for categ in categories:
    p2='{}{}'.format(p1,categ)
    if not os.path.exists(p2):
      #print('{} path does not exist'.format(p2))
      os.makedirs(p2)

In [ ]:
# GENERATE CAMs FOR BASELINE MODEL
categories = ['cat','dog','horse','person']
imgLists = [img_cat, img_dog, img_horse, img_person]

srcDir = './data/test/'
targetDir = './CAMs/'

layerName = 'block5_conv3'
idx = 0
chPruned = 0
prunedLayer='base'
prunedIdx='000'

# NOTE: Project 'm2p01' is the right project for channel pruning
baseModelPath  = './baseModel/modelBasev3.004-0.73.hdf5'
baseModel = build_model(baseModelPath)
guided_baseModel = build_guided_model(baseModelPath)

for categ, imgList in zip(categories, imgLists):
  for srcImg in imgList:
    # Load npy img
    imgPath = '{}{}/{}.jpg'.format(srcDir,categ,srcImg)
    img =load_image(imgPath)
    pred_base = np.round(baseModel.predict(img)[0], decimals=2)
    pred_base = np.array2string(pred_base,separator=',', formatter={'float_kind':lambda x: '{:.2f}'.format(x)})
    pred_base = pred_base.replace('[','(').replace(']',')')

    # Perform {GradCAM, etc.} to explain all target_cls and save images
    for target_cls in [0,1,2,3]:
      p_or_t = 'b'
      imgName = '{}_{}_{:03d}{}_{}_{}_{}_{}.jpg'.format(srcImg,target_cls,idx,p_or_t,prunedLayer, prunedIdx, chPruned, pred_base)

      targetPath_gc  = '{}GradCAM/{}/{}'.format(targetDir,categ, imgName)
      targetPath_ggc = '{}G_GradCAM/{}/{}'.format(targetDir,categ, imgName)
      #targetPath_gb  = '{}G_BackProp/{}/{}'.format(targetDir,categ, imgName)

      gradcam = grad_cam(baseModel, img, target_cls, layerName)
      gb = guided_backprop(guided_baseModel, img, layerName)
      guided_gradcam = gb * gradcam[..., np.newaxis]

      jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
      jetcam = (np.float32(jetcam) + load_image(imgPath, preprocess=False)) / 2

      cv2.imwrite(targetPath_gc, np.uint8(jetcam))
      #cv2.imwrite(targetPath_gb, deprocess_image(gb[0]))
      cv2.imwrite(targetPath_ggc, deprocess_image(guided_gradcam[0]))   

## cmds to build model & guided_model (run once only)

In [ ]:
# Generate cmds to build the model & guided_model()
#===============================
cmds =[]
proj = 'm2p01_' #project name
folderList = [s for s in os.listdir('./models/') if s.startswith(proj)]

for f in folderList:
  files = os.listdir('./models/{}/'.format(f))
  # Ignore empty folders
  if len(files)==0: continue
  
  for file in files:
    
    # Extract info
    chPruned = file.split('.')[2]
    prunedIdx = chPruned[:3]
    chPruned = chPruned[-3:]
    prunedLayer = file.split('.')[1]
    
    if (file.startswith('modelPruned') & file.endswith('.hdf5')):      
      modelName = 'prunedModel'
      cmd1 = 'chPruned = \'{}\'; prunedLayer = \'{}\'; prunedIdx = \'{}\''.format(chPruned, prunedLayer,prunedIdx)
      cmd2 = '{} = build_model(\'./models/{}/{}\')'.format(modelName,f,file)
      cmd3 = 'guided_{} = build_guided_model(\'./models/{}/{}\')'.format(modelName,f,file)
      cmds.append("\n".join([cmd1,cmd2,cmd3]))
    elif (file.startswith('modelTuned') & file.endswith('.hdf5')):
      modelName = 'tunedModel'
      cmd2 = '{} = build_model(\'./models/{}/{}\')'.format(modelName,f,file)
      cmd3 = 'guided_{} = build_guided_model(\'./models/{}/{}\')'.format(modelName,f,file)
      cmds.append("\n".join([cmd2,cmd3]))
      
# Combine the cmds for each pruned-tuned together
cmds_v2=[]
for i in range(0,len(cmds),2):
  cmds_v2.append("\n".join([cmds[i],cmds[i+1]]))
cmds = cmds_v2  
  
  
del files, cmds_v2, folderList, chPruned, prunedIdx, prunedLayer, modelName

In [ ]:
#Save cmds
import pickle
with open("./data/cmds.txt", "wb") as fp:
  pickle.dump(cmds, fp)

## pruned-tuned (cmd 0 to cmd 90)

In [ ]:
# RELOAD BASELINE
categories = ['cat','dog','horse','person']
imgLists = [img_cat, img_dog, img_horse, img_person]

srcDir = './data/test/'
targetDir = './CAMs/'

layerName = 'block5_conv3'
idx = 0
chPruned = 0

# NOTE: Project 'm2p01' is the right project for channel pruning

In [ ]:
import pickle
with open("./data/cmds.txt", "rb") as fp:
  cmds = pickle.load(fp)

del pickle

In [ ]:
# CONTINUE FROM CMD0, idx 1
# ==================
categories = ['cat','dog','horse','person']
imgLists = [img_cat, img_dog, img_horse, img_person]

srcDir = './data/test/'
targetDir = './CAMs/'

layerName = 'block5_conv3'
#################################
contFromCmd = 0
idx = contFromCmd*2+1
#################################

for i,cmd in enumerate(cmds[contFromCmd:], contFromCmd):
  # Load prunedModel, tunedModel, guided_prunedModel, guided_tunedModel
  # AND load chPruned, prunedLayer, prunedIdx
  print('Loading models [cmd {}]'.format(i))
  K.clear_session()
  exec(cmd)
  print('===')
  
  for categ, imgList in zip(categories, imgLists):
    for srcImg in imgList:
      
      # Load npy img
      imgPath = '{}{}/{}.jpg'.format(srcDir,categ,srcImg)
      img =load_image(imgPath)
      pred_pruned = np.round(prunedModel.predict(img)[0], decimals=2)
      pred_pruned = np.array2string(pred_pruned,separator=',', formatter={'float_kind':lambda x: '{:.2f}'.format(x)})
      pred_pruned = pred_pruned.replace('[','(').replace(']',')')
      
      pred_tuned  = np.round(tunedModel.predict(img)[0], decimals=2)
      pred_tuned  = np.array2string(pred_tuned,separator=',', formatter={'float_kind':lambda x: '{:.2f}'.format(x)})
      pred_tuned  = pred_tuned.replace('[','(').replace(']',')')
      
      # Perform GradCAM to explain all target_cls and save images
      for target_cls in [0,1,2,3]:
        p_or_t = 'p'
        imgName = '{}_{}_{:03d}{}_{}_{}_{}_{}.jpg'.format(srcImg,target_cls,idx,p_or_t,prunedLayer, prunedIdx, chPruned,pred_pruned)
        
        targetPath_gc  = '{}GradCAM/{}/{}'.format(targetDir,categ, imgName)
        targetPath_ggc = '{}G_GradCAM/{}/{}'.format(targetDir,categ, imgName)
        #targetPath_gb  = '{}G_BackProp/{}/{}'.format(targetDir,categ, imgName)
        
        gradcam = grad_cam(prunedModel, img, target_cls, layerName)
        gb = guided_backprop(guided_prunedModel, img, layerName)
        guided_gradcam = gb * gradcam[..., np.newaxis]

        jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
        jetcam = (np.float32(jetcam) + load_image(imgPath, preprocess=False)) / 2

        cv2.imwrite(targetPath_gc, np.uint8(jetcam))
        #cv2.imwrite(targetPath_gb, deprocess_image(gb[0]))
        cv2.imwrite(targetPath_ggc, deprocess_image(guided_gradcam[0]))
      print('{}: {}'.format(idx,imgName))
      idx+=1
      
      for target_cls in [0,1,2,3]:
        p_or_t = 't'
        imgName = '{}_{}_{:03d}{}_{}_{}_{}_{}.jpg'.format(srcImg,target_cls,idx,p_or_t,prunedLayer, prunedIdx, chPruned,pred_tuned)

        targetPath_gc  = '{}GradCAM/{}/{}'.format(targetDir,categ, imgName)
        targetPath_ggc = '{}G_GradCAM/{}/{}'.format(targetDir,categ, imgName)
        #targetPath_gb  = '{}G_BackProp/{}/{}'.format(targetDir,categ, imgName)

        gradcam = grad_cam(tunedModel, img, target_cls, layerName)
        gb = guided_backprop(guided_tunedModel, img, layerName)
        guided_gradcam = gb * gradcam[..., np.newaxis]

        jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
        jetcam = (np.float32(jetcam) + load_image(imgPath, preprocess=False)) / 2

        cv2.imwrite(targetPath_gc, np.uint8(jetcam))
        #cv2.imwrite(targetPath_gb, deprocess_image(gb[0]))
        cv2.imwrite(targetPath_ggc, deprocess_image(guided_gradcam[0]))
      print('{}: {}'.format(idx,imgName))
      idx-=1
  idx+=2

# Creating GIF

In [ ]:
def create_gif(filenames, duration, savePath):
    images = []
    for filename in filenames:
        images.append(imread(filename))
    #output_file = 'Gif-%s.gif' % datetime.datetime.now().strftime('%Y-%M-%d-%H-%M-%S')
    #imageio.mimsave(output_file, images, duration=duration)
    mimsave(savePath, images, duration=duration)

In [ ]:
# Make directories if not already exist
categories = ['cat','dog','horse','person']
targetDir = './CAMs/'

for f1 in ['GradCAM_idx', 'G_GradCAM_idx', 'G_BackProp_idx']:
  p1 = '{}{}/'.format(targetDir,f1)
  if not os.path.exists(p1):
    #print('{} path does not exist'.format(p1))
    os.makedirs(p1)
  for categ in categories:
    p2='{}{}'.format(p1,categ)
    if not os.path.exists(p2):
      #print('{} path does not exist'.format(p2))
      os.makedirs(p2)

In [ ]:
# FOR ANNOTATING Images (for GIF creation later)
categories = ['cat','dog','horse','person']

srcDir    = './CAMs/'

for f1 in ['GradCAM', 'G_GradCAM']:
  for categ in categories:
    imgPath = '{}{}/{}/'.format(srcDir, f1, categ)
    files = ['{}{}'.format(imgPath, s) for s in os.listdir(imgPath)]
    
    for file in files:
      #Note cv2 uses BGR instead of RGB >> To be consistent use cv2 to read and write
      img = cv2.imread(file)
      splitFile = file.split('/')
      targetPath = './{}/{}_idx/{}/{}'.format(splitFile[1],splitFile[2],splitFile[3],splitFile[4])
      idx = splitFile[4][15:17]
      if len(idx) ==1: idx=' '+idx
      img = cv2.putText(img, idx, (115, 125), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_8)
      cv2.imwrite(targetPath, img)

In [ ]:
# FOR GRADCAM pruned-tuned GIFs (base->pruned->tuned->pruned->tuned->...)
categories = ['cat','dog','horse','person']
imgLists =[img_cat, img_dog, img_horse, img_person]

srcDir    = './CAMs/GradCAM_idx/'
targetDir = './CAMs/GradCAM_idx/prune-tune GIFs/'

for categ, imgList in zip(categories, imgLists):
  for target_cls in [0,1,2,3]:
    for srcImg in imgList:
      imgPath = '{}{}/'.format(srcDir,categ)
      files = ['{}{}'.format(imgPath, s) for s in os.listdir(imgPath) if s.startswith('{}_{}'.format(srcImg,target_cls))]
      
      splitFile = files[0].split('/')
      targetPath = '{}{}_{}_{}to{}.gif'.format(targetDir,srcImg, target_cls,
                                                 files[0].split('/')[4][:-4][-21:],files[-1].split('/')[4][:-4][-21:])
  
      gifDuration = [0.4 for _ in range(len(files))]
      gifDuration[0] = 3
      gifDuration[-1]= 3
      create_gif(files,gifDuration,targetPath)

In [ ]:
# FOR GRADCAM tuned GIFs (base->tuned->tuned->...)
categories = ['cat','dog','horse','person']
imgLists =[img_cat, img_dog, img_horse, img_person]

srcDir    = './CAMs/GradCAM_idx/'
targetDir = './CAMs/GradCAM_idx/tune GIFs/'

for categ, imgList in zip(categories, imgLists):
  for target_cls in [0,1,2,3]:
    for srcImg in imgList:
      imgPath = '{}{}/'.format(srcDir,categ)
      files = ['{}{}'.format(imgPath, s) for s in os.listdir(imgPath) if s.startswith('{}_{}'.format(srcImg,target_cls)) & (s[17]!='p')]
      splitFile = files[0].split('/')
      targetPath = '{}{}_{}_{}to{}.gif'.format(targetDir,srcImg, target_cls,
                                                 files[0].split('/')[4][:-4][-21:],files[-1].split('/')[4][:-4][-21:])
  
      gifDuration = [0.4 for _ in range(len(files))]
      gifDuration[0] = 3
      gifDuration[-1]= 3
      create_gif(files,gifDuration,targetPath)      


In [ ]:
# FOR Guided-GRADCAM pruned-tuned GIFs (base->pruned->tuned->pruned->tuned->...)
categories = ['cat','dog','horse','person']
imgLists =[img_cat, img_dog, img_horse, img_person]

srcDir    = './CAMs/G_GradCAM_idx/'
targetDir = './CAMs/G_GradCAM_idx/prune-tune GIFs/'

for categ, imgList in zip(categories, imgLists):
  for target_cls in [0,1,2,3]:
    for srcImg in imgList:
      imgPath = '{}{}/'.format(srcDir,categ)
      files = ['{}{}'.format(imgPath, s) for s in os.listdir(imgPath) if s.startswith('{}_{}'.format(srcImg,target_cls))]
      
      splitFile = files[0].split('/')
      targetPath = '{}{}_{}_{}to{}.gif'.format(targetDir,srcImg, target_cls,
                                                 files[0].split('/')[4][:-4][-21:],files[-1].split('/')[4][:-4][-21:])
  
      gifDuration = [0.4 for _ in range(len(files))]
      gifDuration[0] = 3
      gifDuration[-1]= 3
      create_gif(files,gifDuration,targetPath)

In [ ]:
# FOR Guided-GRADCAM tuned GIFs (base->tuned->tuned->...)
categories = ['cat','dog','horse','person']
imgLists =[img_cat, img_dog, img_horse, img_person]

srcDir    = './CAMs/G_GradCAM_idx/'
targetDir = './CAMs/G_GradCAM_idx/tune GIFs/'

for categ, imgList in zip(categories, imgLists):
  for target_cls in [0,1,2,3]:
    for srcImg in imgList:
      imgPath = '{}{}/'.format(srcDir,categ)
      files = ['{}{}'.format(imgPath, s) for s in os.listdir(imgPath) if s.startswith('{}_{}'.format(srcImg,target_cls)) & (s[17]!='p')]
      splitFile = files[0].split('/')
      targetPath = '{}{}_{}_{}to{}.gif'.format(targetDir,srcImg, target_cls,
                                                 files[0].split('/')[4][:-4][-21:],files[-1].split('/')[4][:-4][-21:])
  
      gifDuration = [0.4 for _ in range(len(files))]
      gifDuration[0] = 3
      gifDuration[-1]= 3
      create_gif(files,gifDuration,targetPath)      
